In [1]:
from labpulse.cleaning import prepare_base_df, filter_numeric_rows
from labpulse.qc_rules import apply_unit_rules
from labpulse.alerts import build_outlier_alerts

# 1) Prepare base data
df = prepare_base_df("../data/raw/samples.csv")

# 2) Keep only numeric measurements
df = filter_numeric_rows(df)

# 3) Apply QC unit rules
df = apply_unit_rules(df)

# 4) Build outlier alerts
alerts = build_outlier_alerts(df)

alerts.head(10)


,sample_id,product,parameter,value_num,unit,date_dt,iqr_lower,iqr_upper,severity,severity_level
0,1,Diesel,Ash,0.36,% m/m,2026-01-07,-0.0025,0.0649,17.513353,high
1,94,HSFO,Water,825.00,mg/kg,2026-01-16,10.4150,225.8550,11.124118,high
2,80,HSFO,Water,825.00,mg/kg,2026-01-13,10.4150,225.8550,11.124118,high
3,58,Diesel,Viscosity,810.00,cSt,2026-01-26,34.1000,362.1000,5.462195,high
